# Automated ML

Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
import logging
import os
import csv
import io
import requests
import json

from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from sklearn import datasets
import pkg_resources

import azureml.core
from azureml.core.experiment import Experiment
from azureml.core.workspace import Workspace
from azureml.train.automl import AutoMLConfig
from azureml.core.dataset import Dataset
from azureml.core.webservice import AciWebservice, LocalWebservice
from azureml.core import Environment
from azureml.core.model import InferenceConfig
from azureml.core.model import Model


from azureml.widgets import RunDetails
from azureml.pipeline.steps import AutoMLStep

# Check core SDK version number
print("SDK version:", azureml.core.VERSION)

SDK version: 1.26.0


## Dataset

### Overview
The dataset used was Credit Card Fraud Detection dataset provided in the Kaggle website https://www.kaggle.com/mlg-ulb/creditcardfraud, where the goal is to predict whether a banking transaction would result in fraud based on several features. The dataset was upload to the Azure Blob Storage in the Notebooks section and it can be consumed as follows

In [2]:
ws = Workspace.from_config()

experiment_name = 'fraud_detection'

exp=Experiment(ws, experiment_name)

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()
print(exp)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code F9KKXTU2N to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
Workspace name: quick-starts-ws-143062
Azure region: southcentralus
Subscription id: 1b944a9b-fdae-4f97-aeb1-b7eea0beac53
Resource group: aml-quickstarts-143062
Experiment(Name: fraud_detection,
Workspace: quick-starts-ws-143062)


It was created a Pandas dataframe to explore the dataset

In [3]:
dataset = Dataset.get_by_name(workspace=ws, name='Banking-Transactions')
transactions_df = dataset.to_pandas_dataframe()

This function is defined to explore the dataset and see the proportions of fraudalent and genuine transactions

In [4]:
# Calculate the fraction of data points that are fraudulent
def fraudulent_percentage(transaction_df):
    '''Calculate the fraction of all data points that have a 'Class' label of 1; fraudulent.
       :param transaction_df: Dataframe of all transaction data points; has a column 'Class'
       :return: A fractional percentage of fraudulent data points/all points
    '''
    # counts for all classes
    counts = transactions_df['Class'].value_counts()
    
    # get fraudulent and valid cnts
    fraud_cnts = counts[1]
    valid_cnts = counts[0]
    
    # calculate percentage of fraudulent data
    fraud_percentage = fraud_cnts/(fraud_cnts+valid_cnts)
    
    return fraud_percentage

Here you can see the proportions of fraudalent and genuine transactions

In [5]:
# call the function to calculate the fraud percentage
fraud_percentage = fraudulent_percentage(dataset)

print('Fraudulent percentage = ', fraud_percentage)
print('Total # of fraudulent pts: ', fraud_percentage*transactions_df.shape[0])
print('Out of (total) pts: ', transactions_df.shape[0])

Fraudulent percentage =  0.001727485630620034
Total # of fraudulent pts:  492.0
Out of (total) pts:  284807


In [6]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
# Create compute cluster
# Choose a name for the computer cluster:
amlcompute_cluster_name = "compute-cluster"

# Check if the cluster already exists:
try:
    aml_compute = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print('The cluster already exists')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
    aml_compute = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

aml_compute.wait_for_completion(show_output=True)

Creating....
SucceededProvisioning operation finished, operation "Succeeded"
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


## AutoML Configuration

Automated Machine Learning was used to train the model in the Azure Machine Learning Studio. For this task the Auto ML main configuration consists in binary classification, for the primary metric was chosen AUC weighted since it works well in imbalanced datasets, and the featurization was chosen to be automatic.

In [7]:
# Set the automl settings here
automl_settings = {
    "experiment_timeout_minutes": 20,
    "max_concurrent_iterations": 5,
    "primary_metric" : 'AUC_weighted'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=aml_compute,
                             task = "classification",
                             training_data=dataset,
                             label_column_name="Class",   
                             enable_early_stopping= True,
                             featurization= 'auto',
                             debug_log = "automl_errors.log",
                             **automl_settings)

In [8]:
# Submit the experiment
remote_run = exp.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
fraud_detection,AutoML_ff35158a-806d-424c-9003-feef720d8671,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

Use the `RunDetails` widget to show the different experiments.

In [9]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

## Best Model

Get the best model from the automl experiments and display all the properties of the model.



In [11]:
# Retrieve and save your best automl model.
best_automl_run, best_model = remote_run.get_output()
best_automl_model_name = best_automl_run.properties['model_name']
print(('Best AutoML model name:{}\n\n' \
       'Best AutoML model run:{}\n\n' \
       'Best AutoML model specification:{}').format(best_automl_model_name, str(best_automl_run), str(best_model)))

Best AutoML model name:AutoMLff35158a82

Best AutoML model run:Run(Experiment: fraud_detection,
Id: AutoML_ff35158a-806d-424c-9003-feef720d8671_2,
Type: azureml.scriptrun,
Status: Completed)

Best AutoML model specification:Pipeline(memory=None,
         steps=[('datatransformer',
                 DataTransformer(enable_dnn=None, enable_feature_sweeping=None,
                                 feature_sweeping_config=None,
                                 feature_sweeping_timeout=None,
                                 featurization_config=None, force_text_dnn=None,
                                 is_cross_validation=None,
                                 is_onnx_compatible=None, logger=None,
                                 observer=None, task=None, working_dir=None)),
                ('MinMaxScaler', MinMaxScaler(copy...
                 RandomForestClassifier(bootstrap=False, ccp_alpha=0.0,
                                        class_weight=None, criterion='gini',
                  

In [12]:
registered_model =  remote_run.register_model(model_name = 'best_automl_model')

registered_model.download(target_dir='outputs', exist_ok=True)

'outputs/model.pkl'

## Model Deployment

In the cell below, register the model, create an inference config and deploy the model as a web service.


In [27]:
deployment_config = AciWebservice.deploy_configuration(cpu_cores=1,
                                                memory_gb=4,
                                                enable_app_insights=True,
                                                description='predict a fraud')
model = Model(ws, 'best_automl_model')

env_deploy = Environment.get(workspace=ws, name='AzureML-AutoML')

inference_config = InferenceConfig(entry_script='score.py', environment=env_deploy)

service = Model.deploy(workspace=ws,
                       name='fraud-detection',
                       models=[model],
                       inference_config=inference_config,
                       deployment_config=deployment_config)

In [28]:
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-04-18 19:20:47+00:00 Creating Container Registry if not exists.
2021-04-18 19:20:47+00:00 Registering the environment.
2021-04-18 19:20:48+00:00 Use the existing image.
2021-04-18 19:20:48+00:00 Generating deployment configuration.
2021-04-18 19:20:49+00:00 Submitting deployment to compute.
2021-04-18 19:20:53+00:00 Checking the status of deployment fraud-detection..
2021-04-18 19:24:03+00:00 Checking the status of inference endpoint fraud-detection.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [15]:
print(service.scoring_uri)
print(service.swagger_uri)

http://f27eafae-b45b-4fe4-bd9c-9322cddb2480.southcentralus.azurecontainer.io/score
http://f27eafae-b45b-4fe4-bd9c-9322cddb2480.southcentralus.azurecontainer.io/swagger.json


TODO: In the cell below, send a request to the web service you deployed to test it.

In [20]:
import requests
import json

# URL for the web service, should be similar to:
# 'http://8530a665-66f3-49c8-a953-b82a2d312917.eastus.azurecontainer.io/score'
scoring_uri = service.scoring_uri
# If the service is authenticated, set the key or token
# key = ''



# One set of data to score, so we get two results back
data = {
  "data": [
    {
      "Time": 4,
      "V1": 1.22965763450793,
      "V2": 0.141003507049326,
      "V3": 0.0453707735899449,
      "V4": 1.20261273673594,
      "V5": 0.191880988597645,
      "V6": 0.272708122899098,
      "V7": -0.00515900288250983,
      "V8": 0.0812129398830894,
      "V9": 0.464959994783886,
      "V10": -0.0992543211289237,
      "V11": -1.41690724314928,
      "V12": -0.153825826253651,
      "V13": -0.75106271556262,
      "V14": 0.16737196252175,
      "V15": 0.0501435942254188,
      "V16": -0.443586797916727,
      "V17": 0.00282051247234708,
      "V18": -0.61198733994012,
      "V19": -0.0455750446637976,
      "V20": -0.21963255278686,
      "V21": -0.167716265815783,
      "V22": -0.270709726172363,
      "V23": -0.154103786809305,
      "V24": -0.780055415004671,
      "V25": 0.75013693580659,
      "V26": -0.257236845917139,
      "V27": -0.0345074297438413,
      "V28": 0.00516776890624916,
      "Amount": 4.99
    }
  ]
}
# Convert to JSON string
input_data = json.dumps(data)
with open("data.json", "w") as _f:
    _f.write(input_data)

# Set the content type
headers = {'Content-Type': 'application/json'}
# If authentication is enabled, set the authorization header
# headers['Authorization'] = f'Bearer {key}'

# Make the request and display the response
resp = requests.post(scoring_uri, input_data, headers=headers)
print(resp.json())

[0]


In the cell below, print the logs of the web service and delete the service

In [29]:
print(service.get_logs())

2021-04-18T19:23:50,963184700+00:00 - iot-server/run 
2021-04-18T19:23:50,964556400+00:00 - rsyslog/run 
2021-04-18T19:23:50,998306500+00:00 - nginx/run 
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libcrypto.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
/usr/sbin/nginx: /azureml-envs/azureml_1c91e9a5baadb44a18d64bdd7f599d89/lib/libssl.so.1.0.0: no version information available (required by /usr/sbin/nginx)
2021-04-18T19:23:51,011510600+00:00 - gunicorn/run 
rsyslogd

In [21]:
# delete service
service.delete()